# Model explainability - demonstration of ELI5 on complex sklearn pipelines

## Introduction

The goal of this notebook is to demonstrate how [ELI5](https://github.com/TeamHG-Memex/eli5/)'s model explainability features can be applied to a more complex `sklearn` pipeline. 

For this purpose, we load the Titanic dataset, build a complex `sklearn` pipeline with categorical and numerical features, and then runs a `RandomForestClassifier`.

In particular, we note that complex pipelines don't run out of the box for ELI5 yet, and some manual efforts are needed to get the ELI5 output.

## 1. Initialization

In [1]:
# import packages
import numpy as np
import pandas as pd

## 2. Data Loading

Download titanic dataset. Download [here](https://www.kaggle.com/c/titanic/data), and save in some local data directory.

In [2]:
data_folder = '../../data/titanic/'

In [3]:
X_train, X_test = pd.read_csv(data_folder + 'train.csv'), pd.read_csv(data_folder + 'test.csv')
y_train = X_train.pop('Survived')

## 3. Modelling

### Define a model

Can be anything. Just take a `RandomForestClassifier` here, with some random hyperparameter settings.

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve

rf = RandomForestClassifier(class_weight='balanced_subsample',
                            max_features='sqrt',
                            max_depth=3,
                            n_estimators=100,
                            n_jobs=-1,
                            bootstrap=True,
                            oob_score=True)

### A custom transformer

In real life, you sometimes have to build a custom `sklearn.transformer` for the feature engineering that you would like to do. In this example, let's make a simple one to select dataframe columns based on their column names. This way, we can separate the categorical from the numerical features and apply different feature transformations:

* Turn categorical variables into dummies
* Standard scale numerical variables

The bare minimum for such a transformer would be:

In [5]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

Which we will apply to select the following feature columns:

In [6]:
# categorical variables:
cat_features = ['Pclass', 'Sex', 'Embarked']

# numerical variables
num_features = ['Age', 'SibSp', 'Parch', 'Fare']

### Create the pipeline

Now we have all the components in place, let's build a complex pipeline including our own transformer.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_features)),
    ('imputer', SimpleImputer()),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_features)),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])

feature_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

model_pipeline = Pipeline([
    ('feature_pipeline', feature_pipeline),
    ('classifier', rf)
])

### Training the model

The goal of this specific notebook is to demonstrate model explainability. We thus need to fit one!

In [8]:
clf = model_pipeline.fit(X_train, y_train)

### Getting model scores

Now that we have a fitted model, let's apply it. Outcomes can be looked into by examining the probability scores:

In [9]:
y_pred_proba = clf.predict_proba(X_test)

y_pred_proba[0:5]

array([[0.79913147, 0.20086853],
       [0.42935493, 0.57064507],
       [0.78002511, 0.21997489],
       [0.79637269, 0.20362731],
       [0.34569069, 0.65430931]])

The question now is: "Which features cause a particular score?"

## 4. Model Explainability

To answer this question we can call in external packages. Here we start experimenting with ELI5, but Shapley is another good candidate.

Start by installing the package.

In [10]:
import eli5

### ELI5 introduction and docs

Accordingt to the [ELI5 docs](https://eli5.readthedocs.io/en/latest/overview.html#basic-usage), there are two main ways to look at a classification or a regression model:

1. inspect model parameters and try to figure out how the model works globally;
2. inspect an individual prediction of a model, try to figure out why the model makes the decision it makes.

For (1) ELI5 provides `eli5.show_weights()` function; for (2) it provides `eli5.show_prediction()` function.

Note that depending on what model is used (in our case a `RandomForestClassifier`), ELI5's methods to explain predictions will differ. For now, let's have a look at the section on [sklearn ensambles](https://eli5.readthedocs.io/en/latest/libraries/sklearn.html#decision-trees-ensembles).

Another nice piece of documentation to look into is this [example notebook on `eli5.sklearn`](https://github.com/TeamHG-Memex/eli5/blob/master/notebooks/Debugging%20scikit-learn%20text%20classification%20pipeline.ipynb).

### Global explainability

The functions above don't directly work on our `clf` object. This is because we have a custom component (`DataFrameSelector`) in our model pipeline that doesn't support the required `eli5.transform_feature_names()` method. We can either:
1. Apply the trick to provide only the last step of the pipeline to ELI5 (i.e. the fitted model), or
2. Update our custom component

For now let's continue with option 1, that seems the easiest way to go.

Additionally, for `eli5` to know what feature are in it, we have to provide those through the `feature_names` argument. Sadly, sklearn outputs the categorical features as `x0_, x1_, ...` which isn't very explanatory. So let's first clean up the categorical features names after preprocessing in the `cat_pipeline`: 

In [11]:
replacement_dict = {"x"+str(i):cat_features[i]  for i in range(len(cat_features))}

cat_feature_names = pd.Series(
    model_pipeline.named_steps['feature_pipeline'].transformer_list[1][1].named_steps['cat_encoder']
    .get_feature_names()
).replace(replacement_dict, regex=True).tolist()

feature_names = num_features + cat_feature_names

Now that we have nice feature names, apply ELI5 on the classifier component of our pipeline:

In [12]:
eli5.show_weights(clf.named_steps['classifier'], 
                 feature_names=feature_names, 
                 top=5)

Let's see if this matches the feature importances as provided by `sklearn` itself:

In [13]:
# Get the feature importance for the (last) model

importance = model_pipeline.named_steps['classifier'].feature_importances_

# add feature_names and sort
df_importance = pd.DataFrame(importance,
                             index=feature_names, 
                             columns=["Importance"]).sort_values(by=['Importance'], ascending= False)
df_importance.head()

,Importance
Sex_female,0.327317
Sex_male,0.279569
Fare,0.115303
Pclass_3,0.072795
Pclass_1,0.066178


That seems identical. The cool thing is that `eli5` in addition provides a confidence interval, telling us whether the feature is actually significant or not.

### Individual predictions explainability

To explain an individual prediction, use the `eli5.show_prediction()` function.

Again our pipeline is initially not supported, so if we stick with option 1, we need to do the feature transformation part manually before feeding it into ELI5. At this point it does start feeling as a lot of work, so let's examine option 2 wherein we update our `DataFrameSelector`. 

Carefully reading the [ELI5 documentation](https://eli5.readthedocs.io/en/latest/autodocs/eli5.html#eli5.transform_feature_names) reveals that ELI5 tries to call a `transformer.get_feature_names()` method. So let's include this in our `DataFrameSelector`:

In [14]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        self.feature_names_ = self.attribute_names  # Added for ELI5
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values
    
    # Added for ELI5
    def get_feature_names(self):
        return self.feature_names_


*Note:* the above is purely for demonstration purposes, as `sklego`'s [`ColumnSelector`](https://github.com/koaning/scikit-lego/blob/1d4fd6e8408e5af9d21306b57e3bd3edfd412ff9/sklego/preprocessing/pandastransformers.py#L176) transformer does have the `get_feature_names()` method implemented.


At this point, the pipeline will not run yet, because scikit-learn's `SimpleImputer()` also doesn't have a `get_feature_names()` method! To fix this, ELI5 suggests singledispatching to register `transform_feature_names` for the transformer class in question:

In [15]:
from eli5 import transform_feature_names

@transform_feature_names.register(SimpleImputer)
def get_feature_names(transformer, in_names=None):
    if in_names is None:
        from eli5.sklearn.utils import get_feature_names
        # generate default feature names
        in_names = get_feature_names(transformer, num_features=transformer.n_features_)
    # return a list of strings derived from in_names
    return in_names

Having updated both transformers, redefine and fit our model pipeline with the new `DataFrameSelector` and `SimpleImputer`:

In [16]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_features)),
    ('imputer', SimpleImputer()),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_features)),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])

feature_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

model_pipeline = Pipeline([
    ('feature_pipeline', feature_pipeline),
    ('classifier', rf)
])

clf = model_pipeline.fit(X_train, y_train)

Check if ELI5 works?

In [17]:
eli5.show_weights(clf)

Weight,Feature
0.3429 ± 0.6613,cat_pipeline:x1_female
0.2352 ± 0.5767,cat_pipeline:x1_male
0.1197 ± 0.3283,num_pipeline:Fare
0.1034 ± 0.2716,cat_pipeline:x0_3
0.0569 ± 0.2432,cat_pipeline:x0_1
0.0424 ± 0.0970,num_pipeline:Age
0.0306 ± 0.0862,num_pipeline:SibSp
0.0212 ± 0.0835,cat_pipeline:x2_S
0.0184 ± 0.0620,num_pipeline:Parch
0.0131 ± 0.0675,cat_pipeline:x0_2


Magical! With regards to global explainability of the model, the pipeline names are now automatically included in the feature names! Manual features names can still be provided if you don't like the categorical one hot features names (like we did earlier).

Now let's give the individual predictions a go:

In [18]:
# eli5.show_prediction(clf, X_test.iloc[[0]])

Damn.... It doesn't work...

Googling quite some time, eventually yiels this [Github issue](https://github.com/TeamHG-Memex/eli5/issues/213) where someone flags that `eli5.show_predictions()` doesn't automatically work for large pipelines. So damn, we updated our `DataFrameSelector`, but now we still need to provide input manually...

According to the issue, the error is related to how we handle pandas dataframes. Currently we assume that vectorizer is able to handle a list of inputs as it's input, but this is not correct in this case. A way to make your example work with current ELI5 is to pass an already vectorized document. 

In other words: transform the features first before passing it to the ELI5 function:

In [19]:
# You have to pass an already vectorized document (so transform data by our pipeline process)
eli5.show_prediction(clf.named_steps['classifier'], 
                     clf.named_steps['feature_pipeline'].transform(X_test.iloc[[0]]),
                     feature_names = feature_names, 
                    )

Wihoeee it works!

To get some feeling of which features are important for some random observations, you could sample some numbers and create a generator for it:

In [20]:
from random import sample

sample_size = 10
gen = (i for i in sample(range(len(X_test)), sample_size))

Run the following cell a couple of times to inspect some observations:

In [21]:
sample = next(gen)

print('ELI5 explaination for prediction on observation #{}:'.format(sample))
# You have to pass an already vectorized document (so transform data by our pipeline process)
eli5.show_prediction(clf.named_steps['classifier'], 
                 clf.named_steps['feature_pipeline'].transform(X_test.iloc[[sample]]),
                 feature_names = feature_names, 
                )
    

ELI5 explaination for prediction on observation #223:


Or alternatively, inspect the highest scored observations. These are most likely to churn.

In [22]:
Y_pred_proba = clf.predict_proba(X_test)

pred_proba_1 = [pred[1] for pred in Y_pred_proba]

# create generator for top X highest predictions using argsort:
gen = (i for i in np.argsort(pred_proba_1, )[::-1][:sample_size])

And run this cell to loop through the highest predictions:

In [23]:
sample = next(gen)

print('ELI5 explaination for prediction on observation #{}:'.format(sample))
# You have to pass an already vectorized document (so transform data by our pipeline process)
eli5.show_prediction(clf.named_steps['classifier'], 
                 clf.named_steps['feature_pipeline'].transform(X_test.iloc[[sample]]),
                 feature_names = feature_names, 
                )
    

ELI5 explaination for prediction on observation #218:


Do the values make sense??! Let's look at this example:

In [24]:
X_train.iloc[[sample]]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
218,219,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C


I guess it does, as we know for this dataset that being male lowers your chance of survival, being female increases it (one hot encoded!). Appearantly, `Fare` also contributes quite a lot, but a downside seems to be that the ELI5 contributions don't say why it is contributing: i.e. is it because Fare is high? Or Low? Or is there another pattern that is causing the contribution?

I guess this is up to us to investigate ourselves.

Done.